In [ ]:
%matplotlib inline

In [1]:
!pip3 install pickle5

     |████████████████████████████████| 256 kB 5.5 MB/s 


In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
import pandas as  pd

import random
import os

import pickle5 as pickle
import pandas as pd
import math


In [3]:
class config:
    seed = 42

In [4]:
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore

In [5]:
seed_everything(config.seed)

In [6]:
batch_size = 1

In [7]:
def augment(x,y,size,half_size,data_count,flip_yn,rotation_degree,affine_rate):
    
    
    if flip_yn == 1:
        x_return = x[:, :, :, ::-1]
    else :
        x_return = x    
    if rotation_degree != 0:
        rotation = []
        #회전 좌표 구하기
        for i in range(-half_size,half_size+1,1):
            for j in range(-half_size,half_size+1,1):
                if i*j !=0 :
                    #처음 좌표
                    coor_y = half_size-j
                    coor_x = i+half_size
                    if coor_y > half_size : coor_y = coor_y-1
                    if coor_x > half_size : coor_x = coor_x-1

                    #변경 좌표
                    coor_y_2 = half_size-(i*math.sin(math.pi * (rotation_degree / 180)) + j*math.cos(math.pi * (rotation_degree / 180)))
                    coor_x_2 = (i*math.cos(math.pi * (rotation_degree / 180)) - j*math.sin(math.pi * (rotation_degree / 180)))+half_size
                    if coor_y_2 > half_size : coor_y_2 = coor_y_2-1
                    if coor_x_2 > half_size : coor_x_2 = coor_x_2-1

                    rotation.append([coor_y,coor_x,coor_y_2,coor_x_2])
        data_rotation = pd.DataFrame(rotation)
        data_rotation = data_rotation[(data_rotation[2]>0) & (data_rotation[3]>0)&(data_rotation[2]<size) & (data_rotation[3]<size)]
        x_rotation = np.zeros((data_count,3,size,size))
        for i in range(size) :
            for j in range(size):
                data_rotation_2 = data_rotation[ ((round(data_rotation[2])==i)|(round(data_rotation[2],0)==i)) & ((round(data_rotation[3])==j)|(round(data_rotation[3],0)==j)) ]
                if len(data_rotation_2) > 0:
                    x_rotation[:,:,i,j] = np.mean(x_return[:,:,data_rotation_2[0],data_rotation_2[1]],axis=2)
        x_return = x_rotation

    if affine_rate != 0 :
        pixel = int(size*affine_rate)
        dummy = np.zeros((data_count,3,size,pixel))
        x_return = np.concatenate((dummy,x_return[:,:,:,:-pixel]),axis=3)
        dummy = np.zeros((data_count,3,pixel,size))
        x_return = np.concatenate((dummy,x_return[:,:,:-pixel,:]),axis=2)

    y_return = y
    return x_return,y_return

In [28]:
### transform 설정

transform_array = []
#transform_array.append(transform0)
for i in [0]:
    for j in [0]:
        for k in [0.05,0.15,0.25]:
            transform1  = [i,j,k]  
            transform_array.append(transform1)


In [29]:
with open('/content/drive/MyDrive/imagenet/64/train_data', 'rb') as fo:
    d = pickle.load(fo)
x = d['data']
y = d['labels']  

x_train  =  torch.FloatTensor(x)  
y_train  =  torch.LongTensor(y)

trainset0 = torch.utils.data.TensorDataset(x_train, y_train)
trainloader0 = torch.utils.data.DataLoader(trainset0, batch_size=batch_size, shuffle=False)

trainset_array = []
trainloader_array = []


for i in range(len(transform_array)):
    
    x_augment, y_augment   =  augment(x,y,64,32,len(x),transform_array[i][0],transform_array[i][1],transform_array[i][2])

    x_augment  =  torch.FloatTensor(x_augment.copy())  
    y_augment  =  torch.LongTensor(y_augment)

        
    trainset1 = torch.utils.data.TensorDataset(x_augment, y_augment)
    trainloader1 = torch.utils.data.DataLoader(trainset1, batch_size, shuffle=False)

    trainset_array.append(trainset1)
    trainloader_array.append(trainloader1)



In [10]:
def get_cosine_similarity(x1, x2):
    return (x1 * x2).sum() / ((x1**2).sum()**.5 * (x2**2).sum()**.5)

In [11]:
def image_array_save(image_array,dataloader) :
    for i, data in enumerate(dataloader, 0):
        inputs, labels = data
        image_array.append(inputs[0])
        
        if i == 499 :
            break  

In [30]:
image_array = []

image_0 = [] 
image_array_save(image_0,trainloader0)


image_elements_array = []   
for i in range(len(transform_array)):
    image_elements_array = []    
    image_array_save(image_elements_array,trainloader_array[i])
    image_array.append(image_elements_array)

In [13]:
def cosine_sim(image_1,image_2):
    ret_total = 0
    for i in range(500):
        ret = get_cosine_similarity(image_1[i], image_2[i])
        ret_total = ret_total+ret
    print(ret_total/500)   

In [31]:
for i in range(len(transform_array)):
    print(i)
    cosine_sim(image_0,image_array[i])

0
tensor(0.8789)
1
tensor(0.7305)
2
tensor(0.6052)
